# Train you first forecasting model

So you picked the forecasting project. Good for you! Forecasting is an important skill, and it's helpful to learn about the techchnologies out there that can make your life easier. Or worse, depending on how you look at it. ;)

### 1. Download the Kaggle forecasting data
Use the notebook example for this.

### 2. Access your data

In [1]:
import pandas as pd
import json
import sagemaker
import boto3

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(5113, 99)

In [4]:
df.head()

,Date,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,...,VINI,WASH,WATO,WAUR,WEAT,WEST,WILB,WIST,WOOD,WYNO
0,19940101,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,...,10771800,12116400,11308800,12361800,11331600,10644300,11715600,11241000,10490100,10545300
1,19940102,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,...,4314300,10733400,9154800,12041400,9168300,4082700,9228000,5829900,7412100,3345300
2,19940103,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,...,2976900,11775000,10700400,12687300,11324400,2746500,3686700,4488900,9712200,4442100
3,19940104,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,...,3476400,12159600,11907000,12953100,11903700,2741400,4905000,4089300,11401500,4365000
4,19940105,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,...,6393300,11419500,7334400,10178700,7471500,8235300,11159100,10651500,10006200,8568300


Each row here is a new point in time, and each column is an energy station. That means that each COLUMN is a unique time series data set. We are going to train our first model on a single column. Then, you can extend it by adding more columns.

In [5]:
# build list of locations
columns = list(df.columns[1:])
columns = columns
# columns[:85]
print(columns)

['ACME', 'ADAX', 'ALTU', 'APAC', 'ARNE', 'BEAV', 'BESS', 'BIXB', 'BLAC', 'BOIS', 'BOWL', 'BREC', 'BRIS', 'BUFF', 'BURB', 'BURN', 'BUTL', 'BYAR', 'CAMA', 'CENT', 'CHAN', 'CHER', 'CHEY', 'CHIC', 'CLAY', 'CLOU', 'COOK', 'COPA', 'DURA', 'ELRE', 'ERIC', 'EUFA', 'FAIR', 'FORA', 'FREE', 'FTCB', 'GOOD', 'GUTH', 'HASK', 'HINT', 'HOBA', 'HOLL', 'HOOK', 'HUGO', 'IDAB', 'JAYX', 'KENT', 'KETC', 'LAHO', 'LANE', 'MADI', 'MANG', 'MARE', 'MAYR', 'MCAL', 'MEDF', 'MEDI', 'MIAM', 'MINC', 'MTHE', 'NEWK', 'NINN', 'NOWA', 'OILT', 'OKEM', 'OKMU', 'PAUL', 'PAWN', 'PERK', 'PRYO', 'PUTN', 'REDR', 'RETR', 'RING', 'SALL', 'SEIL', 'SHAW', 'SKIA', 'SLAP', 'SPEN', 'STIG', 'STIL', 'STUA', 'SULP', 'TAHL', 'TALI', 'TIPT', 'TISH', 'VINI', 'WASH', 'WATO', 'WAUR', 'WEAT', 'WEST', 'WILB', 'WIST', 'WOOD', 'WYNO']


In [6]:
target = df['ACME']

In [7]:
target.shape

(5113,)

Great! We have 5113 obervations. That's well over the 300-limit on DeepAR.

### 3. Create Train and Test Sets
Now, we'll build 2 datasets. One for training, another for testing. Both need to be written to json files, then copied over to S3.

In [8]:
def get_split(df, freq='D', split_type = 'train', cols_to_use = ['ACME']):
    rt_set = []
    
    # use 70% for training
    if split_type == 'train':
        lower_bound = 0
        upper_bound = round(df.shape[0] * .7)
        
    # use 15% for validation
    elif split_type == 'validation':
        lower_bound = round(df.shape[0] * .7)
        upper_bound = round(df.shape[0] * .85)
        
    # use 15% for test
    elif split_type == 'test':
        lower_bound = round(df.shape[0] * .85)
        upper_bound = df.shape[0]
            
    # loop through columns you want to use
    for h in list(df):
        if h in cols_to_use:
            
            target_column = df[h].values.tolist()[lower_bound:upper_bound]
            
            date_str = str(df.iloc[0]['Date'])
            
            year = date_str[0:4]
            month = date_str[4:6]
            date = date_str[7:]
                                                
            start_dataset = pd.Timestamp("{}-{}-{} 00:00:00".format(year, month, date, freq=freq))
                        
            # create a new json object for each column
            json_obj = {
                        'start': str(start_dataset),
                        'target': target_column
                       }
    
            rt_set.append(json_obj)
    
    return rt_set

In [9]:
train_set = get_split(df, cols_to_use = columns)
test_set = get_split(df, split_type = 'test', cols_to_use = columns)

In [10]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            # fp.write(json.dumps(data).encode("utf-8"))
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))

In [11]:
write_dicts_to_file('train.json', train_set)
write_dicts_to_file('test.json', test_set)

In [12]:
!aws s3 cp train.json s3://forecasting-do-not-delete/train/train.json
!aws s3 cp test.json s3://forecasting-do-not-delete/test/test.json

upload: ./train.json to s3://forecasting-do-not-delete/train/train.json
upload: ./test.json to s3://forecasting-do-not-delete/test/test.json


### 4. Run a SageMaker Training Job
Ok! If everything worked, we should be able to train a model in SageMaker straight away.

In [13]:
sess = sagemaker.Session()
region = sess.boto_region_name
image = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
role = sagemaker.get_execution_role()   

In [14]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sess,
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='deepar-electricity-demo',
    output_path='s3://forecasting-do-not-delete/output'
)

In [15]:
hyperparameters = {
    
    # frequency interval is once per day
    "time_freq": 'D',
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    
    # let's use the last 30 days for context
    "context_length": str(30),
    
    # let's forecast for 30 days
    "prediction_length": str(30)
}

estimator.set_hyperparameters(**hyperparameters)

In [16]:
data_channels = {
    "train": "s3://forecasting-do-not-delete/train/train.json",
    "test": "s3://forecasting-do-not-delete/test/test.json"
}

estimator.fit(inputs=data_channels, wait=True)

2019-08-01 19:36:37 Starting - Starting the training job.........
2019-08-01 19:37:51 Starting - Launching requested ML instances......
2019-08-01 19:38:55 Starting - Preparing the instances for training......
2019-08-01 19:40:04 Downloading - Downloading input data...
2019-08-01 19:40:25 Training - Downloading the training image.
Arguments: train
[08/01/2019 19:40:42 INFO 140719711954752] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u''}
[08/01/2019 1


2019-08-01 19:40:39 Training - Training image download completed. Training in progress.[08/01/2019 19:40:54 INFO 140719711954752] Epoch[10] Batch[0] avg_epoch_loss=17.143635
[08/01/2019 19:40:54 INFO 140719711954752] #quality_metric: host=algo-1, epoch=10, batch=0 train loss <loss>=17.1436347961
[08/01/2019 19:40:54 INFO 140719711954752] Epoch[10] Batch[5] avg_epoch_loss=17.076578
[08/01/2019 19:40:54 INFO 140719711954752] #quality_metric: host=algo-1, epoch=10, batch=5 train loss <loss>=17.0765784582
[08/01/2019 19:40:54 INFO 140719711954752] Epoch[10] Batch [5]#011Speed: 977.97 samples/sec#011loss=17.076578
[08/01/2019 19:40:54 INFO 140719711954752] processed a total of 579 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1003.0059814453125, "sum": 1003.0059814453125, "min": 1003.0059814453125}}, "EndTime": 1564688454.690512, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688453.687448}

[08/01/2019 19:40:54 IN

[08/01/2019 19:41:04 INFO 140719711954752] processed a total of 584 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1027.496099472046, "sum": 1027.496099472046, "min": 1027.496099472046}}, "EndTime": 1564688464.054054, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688463.026493}

[08/01/2019 19:41:04 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=568.306184732 records/second
[08/01/2019 19:41:04 INFO 140719711954752] #progress_metric: host=algo-1, completed 5 % of epochs
[08/01/2019 19:41:04 INFO 140719711954752] #quality_metric: host=algo-1, epoch=19, train loss <loss>=16.9281686783
[08/01/2019 19:41:04 INFO 140719711954752] best epoch loss so far
[08/01/2019 19:41:04 INFO 140719711954752] Saved checkpoint to "/opt/ml/model/state_838af5ba-6cfa-4d80-8059-774e9945e99d-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 33.89096260070801, "sum": 33.89096260070801

[08/01/2019 19:41:14 INFO 140719711954752] Epoch[29] Batch[5] avg_epoch_loss=16.945883
[08/01/2019 19:41:14 INFO 140719711954752] #quality_metric: host=algo-1, epoch=29, batch=5 train loss <loss>=16.9458827972
[08/01/2019 19:41:14 INFO 140719711954752] Epoch[29] Batch [5]#011Speed: 965.66 samples/sec#011loss=16.945883
[08/01/2019 19:41:14 INFO 140719711954752] processed a total of 621 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1033.9651107788086, "sum": 1033.9651107788086, "min": 1033.9651107788086}}, "EndTime": 1564688474.325652, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688473.291633}

[08/01/2019 19:41:14 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=600.548883745 records/second
[08/01/2019 19:41:14 INFO 140719711954752] #progress_metric: host=algo-1, completed 7 % of epochs
[08/01/2019 19:41:14 INFO 140719711954752] #quality_metric: host=algo-1, epoch=29, train loss <loss>=

[08/01/2019 19:41:24 INFO 140719711954752] Epoch[39] Batch[5] avg_epoch_loss=16.931659
[08/01/2019 19:41:24 INFO 140719711954752] #quality_metric: host=algo-1, epoch=39, batch=5 train loss <loss>=16.9316593806
[08/01/2019 19:41:24 INFO 140719711954752] Epoch[39] Batch [5]#011Speed: 960.63 samples/sec#011loss=16.931659
[08/01/2019 19:41:24 INFO 140719711954752] processed a total of 561 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 957.7929973602295, "sum": 957.7929973602295, "min": 957.7929973602295}}, "EndTime": 1564688484.385087, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688483.426853}

[08/01/2019 19:41:24 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=585.665277384 records/second
[08/01/2019 19:41:24 INFO 140719711954752] #progress_metric: host=algo-1, completed 10 % of epochs
[08/01/2019 19:41:24 INFO 140719711954752] #quality_metric: host=algo-1, epoch=39, train loss <loss>=16

...

[08/01/2019 19:43:19 INFO 140719711954752] processed a total of 564 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 972.4469184875488, "sum": 972.4469184875488, "min": 972.4469184875488}}, "EndTime": 1564688599.291025, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688598.318133}

[08/01/2019 19:43:19 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=579.91697259 records/second
[08/01/2019 19:43:19 INFO 140719711954752] #progress_metric: host=algo-1, completed 38 % of epochs
[08/01/2019 19:43:19 INFO 140719711954752] #quality_metric: host=algo-1, epoch=151, train loss <loss>=16.7025080787
[08/01/2019 19:43:19 INFO 140719711954752] loss did not improve
[08/01/2019 19:43:19 INFO 140719711954752] Epoch[152] Batch[0] avg_epoch_loss=16.804035
[08/01/2019 19:43:19 INFO 140719711954752] #quality_metric: host=algo-1, epoch=152, batch=0 train loss <loss>=16.8040351868
[08/01/2019 19:43:20 INFO 140719

[08/01/2019 19:43:29 INFO 140719711954752] processed a total of 618 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1016.5529251098633, "sum": 1016.5529251098633, "min": 1016.5529251098633}}, "EndTime": 1564688609.211519, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688608.19491}

[08/01/2019 19:43:29 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=607.865725848 records/second
[08/01/2019 19:43:29 INFO 140719711954752] #progress_metric: host=algo-1, completed 40 % of epochs
[08/01/2019 19:43:29 INFO 140719711954752] #quality_metric: host=algo-1, epoch=161, train loss <loss>=16.7578994751
[08/01/2019 19:43:29 INFO 140719711954752] loss did not improve
[08/01/2019 19:43:29 INFO 140719711954752] Epoch[162] Batch[0] avg_epoch_loss=16.791227
[08/01/2019 19:43:29 INFO 140719711954752] #quality_metric: host=algo-1, epoch=162, batch=0 train loss <loss>=16.7912273407
[08/01/2019 19:43:29 INFO 140

[08/01/2019 19:43:44 INFO 140719711954752] Epoch[176] Batch[5] avg_epoch_loss=16.751077
[08/01/2019 19:43:44 INFO 140719711954752] #quality_metric: host=algo-1, epoch=176, batch=5 train loss <loss>=16.7510773341
[08/01/2019 19:43:44 INFO 140719711954752] Epoch[176] Batch [5]#011Speed: 959.61 samples/sec#011loss=16.751077
[08/01/2019 19:43:44 INFO 140719711954752] processed a total of 606 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1040.6842231750488, "sum": 1040.6842231750488, "min": 1040.6842231750488}}, "EndTime": 1564688624.715447, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688623.674323}

[08/01/2019 19:43:44 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=582.246108335 records/second
[08/01/2019 19:43:44 INFO 140719711954752] #progress_metric: host=algo-1, completed 44 % of epochs
[08/01/2019 19:43:44 INFO 140719711954752] #quality_metric: host=algo-1, epoch=176, train loss <l

[08/01/2019 19:43:54 INFO 140719711954752] Epoch[186] Batch[0] avg_epoch_loss=16.666842
[08/01/2019 19:43:54 INFO 140719711954752] #quality_metric: host=algo-1, epoch=186, batch=0 train loss <loss>=16.666841507
[08/01/2019 19:43:54 INFO 140719711954752] Epoch[186] Batch[5] avg_epoch_loss=16.711749
[08/01/2019 19:43:54 INFO 140719711954752] #quality_metric: host=algo-1, epoch=186, batch=5 train loss <loss>=16.7117490768
[08/01/2019 19:43:54 INFO 140719711954752] Epoch[186] Batch [5]#011Speed: 956.84 samples/sec#011loss=16.711749
[08/01/2019 19:43:54 INFO 140719711954752] processed a total of 621 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1017.7750587463379, "sum": 1017.7750587463379, "min": 1017.7750587463379}}, "EndTime": 1564688634.983874, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688633.965608}

[08/01/2019 19:43:54 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=610.09158508 r

[08/01/2019 19:44:09 INFO 140719711954752] processed a total of 582 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1036.412000656128, "sum": 1036.412000656128, "min": 1036.412000656128}}, "EndTime": 1564688649.227472, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564688648.190627}

[08/01/2019 19:44:09 INFO 140719711954752] #throughput_metric: host=algo-1, train throughput=561.488803967 records/second
[08/01/2019 19:44:09 INFO 140719711954752] #progress_metric: host=algo-1, completed 50 % of epochs
[08/01/2019 19:44:09 INFO 140719711954752] #quality_metric: host=algo-1, epoch=200, train loss <loss>=16.7900470734
[08/01/2019 19:44:09 INFO 140719711954752] loss did not improve
[08/01/2019 19:44:09 INFO 140719711954752] Loading parameters from best epoch (160)
#metrics {"Metrics": {"state.deserialize.time": {"count": 1, "max": 10.210037231445312, "sum": 10.210037231445312, "min": 10.210037231445312}}, "EndTime": 1564

# 5. Run Inference
If you made it this far, congratulations! None of this is easy. For your next steps, please open up the example notebook under the SageMakerExamples:

- SageMakerExamples/Introduction To Amazon Algorithms/DeepAR-Electricity.
- **Guys I found this online - not sure if this is the right thing or not but wanted to add it here in case we don't get to it before I have to leave**
    - https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/deepar_electricity/DeepAR-Electricity.ipynb

That will walk you through both how to add more timeseries to your model, and how to get inference results out of it.

### 6. Extend Your Solution
Now you're getting forecasts, how will you extend your solution? How good are your forecasts? What about getting forecasts for the other stations? Is your model cognizant of the weather?

Spend your remaining time growing your modeling solution to leverage additional datasets. Then, think through how you'd set this up to run in production.